In [10]:
!pip install torch datasets transformers

In [11]:
!pip install peft
!pip uninstall aiohttp -y
!pip cache purge
!pip install --upgrade pip
!rm -rf /opt/conda/lib/python3.10/site-packages/aiohttp*
!pip install aiohttp
!pip install trl


Found existing installation: aiohttp 3.9.5
Uninstalling aiohttp-3.9.5:
  Successfully uninstalled aiohttp-3.9.5
Files removed: 8
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.0 MB/s eta 0:00:00a 0:00:01


In [24]:
from datasets import load_dataset
dataset = load_dataset("deepmind/code_contests")


Resolving data files:   0%|          | 0/39 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

In [25]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['name', 'description', 'public_tests', 'private_tests', 'generated_tests', 'source', 'difficulty', 'solutions', 'incorrect_solutions', 'cf_contest_id', 'cf_index', 'cf_points', 'cf_rating', 'cf_tags', 'is_description_translated', 'untranslated_description', 'time_limit', 'memory_limit_bytes', 'input_file', 'output_file'],
        num_rows: 13328
    })
    test: Dataset({
        features: ['name', 'description', 'public_tests', 'private_tests', 'generated_tests', 'source', 'difficulty', 'solutions', 'incorrect_solutions', 'cf_contest_id', 'cf_index', 'cf_points', 'cf_rating', 'cf_tags', 'is_description_translated', 'untranslated_description', 'time_limit', 'memory_limit_bytes', 'input_file', 'output_file'],
        num_rows: 165
    })
    valid: Dataset({
        features: ['name', 'description', 'public_tests', 'private_tests', 'generated_tests', 'source', 'difficulty', 'solutions', 'incorrect_solutions', 'cf_contest_id', 'cf_ind

In [26]:
dataset['train'][0]

{'name': 'brcktsrm',
 'description': 'Problem description.\nVipul is a hardworking super-hero who maintains the bracket ratio of all the strings in the world. Recently he indulged himself in saving the string population so much that he lost his ability for checking brackets (luckily, not permanently ).Being his super-hero friend\xa0help him in his time of hardship. \n\nInput\n\nThe first line of the input contains an integer T denoting the number of test cases. The description of T test cases follows.\nThe first line of each test case contains a single string S denoting the string to be checked.\n\n\nOutput\n\nFor each test case, output a single line printing "YES" or "NO" (without " " and in uppercase only) , denoting if the brackets in the given string is balanced or not .\n\n\nConstraints\n\n1 ≤ T ≤ 10\n1 ≤ length of S ≤ 60\n\n\nExample\nInput:\n3\n((()))\n(())()\n()(()\n\nOutput:\nYES\nYES\nNO\n\n\xa0\n\nExplanation\nExample is self-explanatory.',
 'public_tests': {'input': ['3\n((

In [27]:
# def create_prompt(dataset):
#     reformatted_segments = []
#     prompt = f"""Instruction: Please analyze the problem description and generate test cases based on the provided details.
# Name: {dataset['name']}:
# {dataset['description']}

#   Public_Tests:
#   {dataset['public_tests']}

#   Output: {dataset['generated_tests']}
#     """
#     reformatted_segments.append(f'<s>[INST]{prompt}[/INST]</s>')
#     return {'input': ''.join(reformatted_segments)}

In [28]:
def create_prompt(data):
    return {
        'input_text': f"{data['name']} {data['description']} {data['public_tests']}",
        'target_text': data['generated_tests']
    }

In [29]:
original_columns = dataset['train'].column_names
original_columns

['name',
 'description',
 'public_tests',
 'private_tests',
 'generated_tests',
 'source',
 'difficulty',
 'solutions',
 'incorrect_solutions',
 'cf_contest_id',
 'cf_index',
 'cf_points',
 'cf_rating',
 'cf_tags',
 'is_description_translated',
 'untranslated_description',
 'time_limit',
 'memory_limit_bytes',
 'input_file',
 'output_file']

In [30]:
dataset['train'] = dataset['train'].map(
    create_prompt,
    remove_columns=original_columns
)
dataset['test'] = dataset['test'].map(
    create_prompt,
    remove_columns=original_columns
)
dataset['valid'] = dataset['valid'].map(
    create_prompt,
    remove_columns=original_columns
)


Map:   0%|          | 0/13328 [00:00<?, ? examples/s]

Map:   0%|          | 0/165 [00:00<?, ? examples/s]

Map:   0%|          | 0/117 [00:00<?, ? examples/s]

In [31]:
print(dataset['train'][0])

{'input_text': 'brcktsrm Problem description.\nVipul is a hardworking super-hero who maintains the bracket ratio of all the strings in the world. Recently he indulged himself in saving the string population so much that he lost his ability for checking brackets (luckily, not permanently ).Being his super-hero friend\xa0help him in his time of hardship. \n\nInput\n\nThe first line of the input contains an integer T denoting the number of test cases. The description of T test cases follows.\nThe first line of each test case contains a single string S denoting the string to be checked.\n\n\nOutput\n\nFor each test case, output a single line printing "YES" or "NO" (without " " and in uppercase only) , denoting if the brackets in the given string is balanced or not .\n\n\nConstraints\n\n1 ≤ T ≤ 10\n1 ≤ length of S ≤ 60\n\n\nExample\nInput:\n3\n((()))\n(())()\n()(()\n\nOutput:\nYES\nYES\nNO\n\n\xa0\n\nExplanation\nExample is self-explanatory. {\'input\': [\'3\\n((()))\\n(())()\\n()(()\'], \'

In [33]:
print(dataset['train'][0]['input_text'])

brcktsrm Problem description.
Vipul is a hardworking super-hero who maintains the bracket ratio of all the strings in the world. Recently he indulged himself in saving the string population so much that he lost his ability for checking brackets (luckily, not permanently ).Being his super-hero friend help him in his time of hardship. 

Input

The first line of the input contains an integer T denoting the number of test cases. The description of T test cases follows.
The first line of each test case contains a single string S denoting the string to be checked.


Output

For each test case, output a single line printing "YES" or "NO" (without " " and in uppercase only) , denoting if the brackets in the given string is balanced or not .


Constraints

1 ≤ T ≤ 10
1 ≤ length of S ≤ 60


Example
Input:
3
((()))
(())()
()(()

Output:
YES
YES
NO

 

Explanation
Example is self-explanatory. {'input': ['3\n((()))\n(())()\n()(()'], 'output': ['YES\nYES\nNO']}


# Working with Model

In [35]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

2024-06-07 06:31:51.968625: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-07 06:31:51.968808: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-07 06:31:52.123316: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [36]:

from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("codellama/CodeLlama-7b-hf",trust_remote_code= True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [37]:
model = AutoModelForCausalLM.from_pretrained(
    "codellama/CodeLlama-7b-hf")

config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [38]:
from torchinfo import summary
summary(model)


Layer (type:depth-idx)                                  Param #
LlamaForCausalLM                                        --
├─LlamaModel: 1-1                                       --
│    └─Embedding: 2-1                                   131,137,536
│    └─ModuleList: 2-2                                  --
│    │    └─LlamaDecoderLayer: 3-1                      202,383,360
│    │    └─LlamaDecoderLayer: 3-2                      202,383,360
│    │    └─LlamaDecoderLayer: 3-3                      202,383,360
│    │    └─LlamaDecoderLayer: 3-4                      202,383,360
│    │    └─LlamaDecoderLayer: 3-5                      202,383,360
│    │    └─LlamaDecoderLayer: 3-6                      202,383,360
│    │    └─LlamaDecoderLayer: 3-7                      202,383,360
│    │    └─LlamaDecoderLayer: 3-8                      202,383,360
│    │    └─LlamaDecoderLayer: 3-9                      202,383,360
│    │    └─LlamaDecoderLayer: 3-10                     202,383,360
│    │    └

In [46]:
import torch

def generate_output(model, tokenizer, prompt):
    # Tokenize the prompt
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    
    # Generate output
    with torch.no_grad():
        output_ids = model.generate(input_ids, 
                                    max_length=2048,  # Allow varying length
                                    num_return_sequences=1,
                                    attention_mask=input_ids.ne(tokenizer.pad_token_id),  # Pass attention mask
                                    pad_token_id=tokenizer.pad_token_id,  # Set pad token ID
                                    temperature=0.8)  # Set temperature

    # Decode the output and return it
    generated_output = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated_output


In [ ]:
#not able to generate any output before finetuning , just generating some garbage value , it Hallucinating
generate_output(model,tokenizer, 'what is deep learning?',)

## Setting things for the Fine tuning the Model

In [ ]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training
peft_config = LoraConfig(
    lora_alpha = 32,
    lora_dropout = 0.1,
    r = 8 ,
    bias = "none",
    task_type = "CAUSAL_LM"
)

In [ ]:
model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8) #for the quantization
model = get_peft_model(model, peft_config)

In [ ]:
class SaveLogsCallback(TrainerCallback):
    def on_epoch_end(self, args, state, control, logs=None, **kwargs):
        # Get the current epoch number
        epoch = state.epoch

        # Save the logs to the specified directory
        logs_dir = os.path.join(args.logging_dir, f"epoch_{epoch}_logs.txt")
        with open(logs_dir, "w") as f:
            f.write(str(logs))

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="./codellama_testcase_generation",
    num_train_epochs=20,
    per_device_train_batch_size=4,
    warmup_steps=0.03,
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="steps",
    eval_steps=20,
    learning_rate=2e-4,
    lr_scheduler_type='constant',
     logging_dir="./logs_generation",
)


In [ ]:
max_seq_length = None
trainer = SFTTrainer(
    model=model,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['valid'],
    dataset_text_field='input_text',
    callbacks=[SaveLogsCallback()]
)


In [ ]:
trainer.train()

In [ ]:
trainer.model.save_pretrained("code_lamma_testcase_generation")

## **Code for checking the model performance:**

In [ ]:
results = trainer.evaluate()
print(results)

